__Aim:__ Consistency analysis across cell lines using pathway enrichment results to find commonly up/down regulated pathways and their genes. 

___
- https://github.com/abearab/ipage-msigdb
- https://github.com/tanghaibao/goatools
- https://gseapy.readthedocs.io/en/latest/index.html

Hani: _"You must redo iPAGE with `--independence=0` option for comparing results between multiple smaples."_

In [1]:
import re
import sys
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from time import time 
from glob import glob
from matplotlib.pyplot import subplots
from itertools import chain
from functools import reduce
from venn import venn

# sys.path.append('/rumi/shams/abe/Projects/ipage-down/')
sys.path.append('/data_gilbert/home/aarab/Projects/pager/')

import upsetplot as us
import ipage_down as ipd
from matplotlib.backends.backend_pdf import PdfPages

from IPython.display import IFrame

In [2]:
pagerdir='/data_gilbert/home/aarab/Projects/pager'

In [3]:
# def get_pvmatrix_list(parent_path,pattern):
#     """
#     pattern: msigdb gene set cluster name 
#     """
#     return glob(f'{parent_path}/*{pattern}*/pvmatrix.txt')

# def get_pv_signal(exp,patterns,pdf=None):
#     """
#     exp: experiment name
#     patterns: list of patterns
#     """
#     cols = pd.concat([
#         ipd.pvmat2bio_signal(ipd.merge_multiple_pvmat(
#             get_pvmatrix_list(f"{exp}_fdr","c1")
#         ),s,n_clust=1) for s in ['up','both','down']
#     ]).columns
    
#     pv_signal = pd.concat([
#         ipd.pvmat2bio_signal(ipd.merge_multiple_pvmat([
#             pvmat for pattern in patterns
#             for pvmat in 
#             get_pvmatrix_list(f"{exp}",pattern) + \
#             get_pvmatrix_list(f"{exp}_fdr",pattern)
#         ]),s,n_clust=2) for s in ['up','both','down']
#     ])
    
#     pv_signal.columns = cols
#     pv_signal=pv_signal[~pv_signal.duplicated()]
    
#     if pdf:
#         pv_signal.to_csv('temp-pvmatrix.txt',sep='\t')
    
#         !bash {pager_dir}/ipage_draw_matrix.sh \
#             {exp}'.txt' "temp-pvmatrix.txt" \
#             {pdf} &> /dev/null
    
#         !rm -v 'temp-pvmatrix.txt'
#     else: 
#         return pv_signal


In [3]:
dictfilt = lambda x, y: dict([ (i,x[i]) for i in x if i in set(y) ])

In [8]:
sys.path[-1]

'/data_gilbert/home/aarab/Projects/pager/'

In [9]:
!cat /data_gilbert/home/aarab/Projects/pager/ipage_loop.sh

## $1: input file contain table with first two columns as gene name/id and numeric value
# pattern='msigdb_v0.3_m*'
pattern='msigdb_v7*'


expfile=`basename $1`
outdir=${1/.txt/};

mkdir -p $outdir
cd $outdir; cd ../

for f in `ls -d $PAGEDIR/PAGE_DATA/ANNOTATIONS/${pattern}`; do
    base=`basename "$f"`;
    echo '________________' $base '________________';
    if [ -d "${outdir}/${base}/" ]; 
    then
        echo 'This result exist!';
    else
        # Run iPAGE 
        perl $PAGEDIR/page.pl --expfile=$expfile --species=$base --exptype=continuous --ebins=11 --nodups=1 --independence=0;
            # --independence=0; option for comparing results between multiple smaples.
        wait
        mv -v ${expfile}_PAGE/ ${outdir}/${base}/;
        # remove the result folder if it was empty
        counter="$(wc -l < ${outdir}/${base}/pvmatrix.txt)"
        if [ $counter -le "$(echo '1')" ] || [ -z $counter ]
        then
            rm -r ${outdir}/${base}
        fi;
        rm -r \
  

## $\Delta$Stability

In [ ]:
%%bash
export PAGEDIR='/data_gilbert/home/aarab/iPAGE'

cd stbl/
ls *_delta_stbl.txt | parallel -j18 -k bash /data_gilbert/home/aarab/Projects/pager/ipage_loop.sh
cd ../

### Remake from pvmatrix (correct bin clusters)

### HL-60 time series

In [218]:
# hl60_pws ['120h'] = merge_multiple_pvmat(
#     pvmat_list = glob('exp/hl60_120h_delta_exp_ind0/*c5*/pvmatrix.[L-R].txt')
# )

#### hl60 6h delta_stbl

In [ ]:
pagedir

'/data_gilbert/home/aarab/iPAGE'

In [43]:
study = 'hl60_6h_delta_stbl'

pvmat = ipd.merge_multiple_pvmat(
    pvmat_list = glob(f'stbl/{study}/*c5.go*/pvmatrix.txt') #+ glob(f'stbl/{study}_ind1/*c2.cp.kegg*/pvmatrix.txt')
)

bio_signal = pd.concat([
    ipd.pvmat2bio_signal(pvmat,side='down',n_clust=1),
    ipd.pvmat2bio_signal(pvmat,side='up',n_clust=1),
    ipd.pvmat2bio_signal(pvmat,side='both'),
],axis=0)

bio_signal.to_csv(f'stbl/{study}_pvmatrix.txt',sep='\t')

exp_file = f'{study}.txt'
pvmat_file = f'{study}_pvmatrix.txt'
pdf_file = f'{study}.pdf'

!cd stbl; bash {pagerdir}/ipage_draw_matrix.sh $exp_file $pvmat_file $pdf_file &> /dev/null; cd ../

In [44]:
pdf = 'stbl/hl60_6h_delta_stbl.pdf'
IFrame(pdf, width=800, height=200)

In [46]:
# gs = 'GOBP_REGULATION_OF_CHOLESTEROL_BIOSYNTHETIC_PROCESS'
# pvmat_list = glob(f'stbl/{study}_ind0/*c5.go*/pvmatrix.txt')
# gs_cluster_path = ipd.detect_gs_cluster(pvmat_list, gs=gs)

# print ([p.split('/')[1:3] for p in gs_cluster_path])

# gs_cluster_path = gs_cluster_path[1].split('pvmatrix.txt')[0]
# print ('Down')
# print (ipd.bin_identifier_genes(gs_cluster_path, '0', gs=gs))
# print ('Up')
# print (ipd.bin_identifier_genes(gs_cluster_path, '9', gs=gs))
# print (ipd.bin_identifier_genes(gs_cluster_path, '10', gs=gs))

#### hl60 72h delta_stbl

In [47]:
study = 'hl60_72h_delta_stbl'

pvmat = ipd.merge_multiple_pvmat(
    pvmat_list = glob(f'stbl/{study}/*c5.go*/pvmatrix.txt') #+ glob(f'stbl/{study}_ind1/*c2.cp.kegg*/pvmatrix.txt')
)

bio_signal = pd.concat([
    ipd.pvmat2bio_signal(pvmat,side='down',n_clust=1),
    ipd.pvmat2bio_signal(pvmat,side='up',n_clust=1),
    ipd.pvmat2bio_signal(pvmat,side='both'),
],axis=0)

bio_signal.to_csv(f'stbl/{study}_pvmatrix.txt',sep='\t')

exp_file = f'{study}.txt'
pvmat_file = f'{study}_pvmatrix.txt'
pdf_file = f'{study}.pdf'

!cd stbl; bash {pagerdir}/ipage_draw_matrix.sh $exp_file $pvmat_file $pdf_file &> /dev/null; cd ../

In [48]:
pdf = 'stbl/hl60_72h_delta_stbl.pdf'
IFrame(pdf, width=800, height=200)

In [19]:
# study = 'hl60_72h_delta_stbl'
study = 'hl60_120h_delta_stbl'

pvmat = ipd.merge_multiple_pvmat(
    pvmat_list = glob(f'stbl/{study}/*c3*/pvmatrix.txt') 
    #+ glob(f'stbl/{study}_ind1/*c2.cp.kegg*/pvmatrix.txt')
)

bio_signal = pd.concat([
    ipd.pvmat2bio_signal(pvmat,side='down',n_clust=1),
    ipd.pvmat2bio_signal(pvmat,side='up',n_clust=1),
    ipd.pvmat2bio_signal(pvmat,side='both'),
],axis=0)

bio_signal

,[-0.44 -0.36],[-0.36 -0.20],[-0.20 -0.13],[-0.13 -0.07],[-0.07 -0.01],[-0.01 0.03],[0.03 0.07],[0.07 0.13],[0.13 0.20],[0.20 0.32],[0.32 0.4]
METTL14_TARGET_GENES,3.020,0.282,-0.644,0.539,0.539,-0.321,-1.250,0.539,-1.250,0.282,-1.239
RACTNNRTTTNC_UNKNOWN,2.748,1.444,-0.872,0.528,-0.872,0.236,-0.378,0.236,-0.872,-0.872,-0.386
CCAATNNSNNNGCG_UNKNOWN,0.216,0.490,-0.407,0.490,0.216,-0.915,-0.915,-0.915,-0.915,0.490,3.388
MIR4666B,-0.832,-0.352,-0.832,-0.352,0.255,-0.832,-0.832,0.993,0.255,0.566,2.906
MIR193A_3P_MIR193B_3P,0.278,-0.325,-0.790,-0.790,-0.790,-0.790,0.278,0.278,-0.325,1.620,2.300


In [22]:
gs = 'MIR193A_3P_MIR193B_3P'
pvmat_list = glob(f'stbl/{study}/*c3*/pvmatrix.txt')
gs_cluster_path = ipd.detect_gs_cluster(pvmat_list, gs=gs)

print ([p.split('/')[1:3] for p in gs_cluster_path])

gs_cluster_path = gs_cluster_path[0].split('pvmatrix.txt')[0]
print ('Down')
print (ipd.bin_identifier_genes(gs_cluster_path, '0', gs=gs))
# print (ipd.bin_identifier_genes(gs_cluster_path, '1', gs=gs))
# print (ipd.bin_identifier_genes(gs_cluster_path, '2', gs=gs))
# print (ipd.bin_identifier_genes(gs_cluster_path, '3', gs=gs))
print ('Up')
print (ipd.bin_identifier_genes(gs_cluster_path, '9', gs=gs))
print (ipd.bin_identifier_genes(gs_cluster_path, '10', gs=gs))

[['hl60_120h_delta_stbl', 'msigdb_v7.4_c3.mir.mirdb']]
Down
{'MIR193A_3P_MIR193B_3P': ['NF2', 'ADARB1']}
Up
{'MIR193A_3P_MIR193B_3P': ['NAV3', 'SOS2', 'DMXL2', 'CLOCK', 'TAPT1']}
{'MIR193A_3P_MIR193B_3P': ['CNOT6', 'INO80D', 'AAK1', 'LAMC1', 'PLXNC1', 'CREBRF']}


#### hl60 120h delta_stbl

In [51]:
study = 'hl60_120h_delta_stbl'

pvmat = ipd.merge_multiple_pvmat(
    pvmat_list = glob(f'stbl/{study}/*c5.go*/pvmatrix.txt') #+ glob(f'stbl/{study}_ind1/*c2.cp.kegg*/pvmatrix.txt')
)

bio_signal = pd.concat([
    ipd.pvmat2bio_signal(pvmat,side='down',n_clust=1),
    ipd.pvmat2bio_signal(pvmat,side='up',n_clust=1),
    ipd.pvmat2bio_signal(pvmat,side='both'),
],axis=0)

bio_signal.to_csv(f'stbl/{study}_pvmatrix.txt',sep='\t')

exp_file = f'{study}.txt'
pvmat_file = f'{study}_pvmatrix.txt'
pdf_file = f'{study}.pdf'

!cd stbl; bash {pagerdir}/ipage_draw_matrix.sh $exp_file $pvmat_file $pdf_file &> /dev/null; cd ../

In [52]:
pdf = 'stbl/hl60_120h_delta_stbl.pdf'
IFrame(pdf, width=800, height=200)

In [54]:
# gs = 'GOBP_RESPONSE_TO_STEROL'
# pvmat_list = glob(f'stbl/{study}_ind0/*c5.go*/pvmatrix.txt')
# gs_cluster_path = ipd.detect_gs_cluster(pvmat_list, gs=gs)

# print ([p.split('/')[1:3] for p in gs_cluster_path])

# gs_cluster_path = gs_cluster_path[1].split('pvmatrix.txt')[0]
# print ('Down')
# print (ipd.bin_identifier_genes(gs_cluster_path, '0', gs=gs))
# print ('Up')
# print (ipd.bin_identifier_genes(gs_cluster_path, '9', gs=gs))
# print (ipd.bin_identifier_genes(gs_cluster_path, '10', gs=gs))

In [55]:
# ind1 = 'hl60_120h_delta_stbl_ind1'
# ipd.clean_bins_range(
#     pd.concat([
#         ipd.read_pvmatrix(f'stbl/{ind1}/msigdb_v7.4_c5.all/pvmatrix.L.txt'),
#         ipd.read_pvmatrix(f'stbl/{ind1}/msigdb_v7.4_c5.all/pvmatrix.R.txt')
#     ])
# ).drop_duplicates().to_csv(f"stbl/{ind1.replace('_ind1','_pvmatrix')}.txt",sep = '\t')

#### molm14_delta_stbl

In [56]:
study = 'molm14_delta_stbl'

pvmat = ipd.merge_multiple_pvmat(
    pvmat_list = glob(f'stbl/{study}/*c5.go*/pvmatrix.txt') #+ glob(f'stbl/{study}_ind1/*c2.cp.kegg*/pvmatrix.txt')
)

bio_signal = pd.concat([
    ipd.pvmat2bio_signal(pvmat,side='down',n_clust=1),
    ipd.pvmat2bio_signal(pvmat,side='up',n_clust=1),
    ipd.pvmat2bio_signal(pvmat,side='both'),
],axis=0)

bio_signal.to_csv(f'stbl/{study}_pvmatrix.txt',sep='\t')

exp_file = f'{study}.txt'
pvmat_file = f'{study}_pvmatrix.txt'
pdf_file = f'{study}.pdf'

!cd stbl; bash {pagerdir}/ipage_draw_matrix.sh $exp_file $pvmat_file $pdf_file &> /dev/null; cd ../

In [57]:
pdf = 'stbl/molm14_delta_stbl.pdf'
IFrame(pdf, width=800, height=200)

#### ociaml2_delta_stbl

In [61]:
study = 'ociaml2_delta_stbl'

pvmat = ipd.merge_multiple_pvmat(
    pvmat_list = glob(f'stbl/{study}/*c5.go*/pvmatrix.txt') #+ glob(f'stbl/{study}_ind1/*c2.cp.kegg*/pvmatrix.txt')
)

bio_signal = pd.concat([
    ipd.pvmat2bio_signal(pvmat,side='down',n_clust=1),
    ipd.pvmat2bio_signal(pvmat,side='up',n_clust=1),
    ipd.pvmat2bio_signal(pvmat,side='both'),
],axis=0)

bio_signal.to_csv(f'stbl/{study}_pvmatrix.txt',sep='\t')

exp_file = f'{study}.txt'
pvmat_file = f'{study}_pvmatrix.txt'
pdf_file = f'{study}.pdf'

!cd stbl; bash {pagerdir}/ipage_draw_matrix.sh $exp_file $pvmat_file $pdf_file &> /dev/null; cd ../

In [62]:
pdf = 'stbl/ociaml2_delta_stbl.pdf'
IFrame(pdf, width=800, height=200)

#### ociaml3_delta_stbl

In [72]:
study = 'ociaml3_delta_stbl'

pvmat = ipd.merge_multiple_pvmat(
    pvmat_list = glob(f'stbl/{study}/*c5.go*/pvmatrix.txt') #+ glob(f'stbl/{study}_ind1/*c2.cp.kegg*/pvmatrix.txt')
)

bio_signal = pd.concat([
    ipd.pvmat2bio_signal(pvmat,side='down',n_clust=1),
    ipd.pvmat2bio_signal(pvmat,side='up',n_clust=1),
    ipd.pvmat2bio_signal(pvmat,side='both'),
],axis=0)

bio_signal.to_csv(f'stbl/{study}_pvmatrix.txt',sep='\t')

exp_file = f'{study}.txt'
pvmat_file = f'{study}_pvmatrix.txt'
pdf_file = f'{study}.pdf'

!cd stbl; bash {pagerdir}/ipage_draw_matrix.sh $exp_file $pvmat_file $pdf_file &> /dev/null; cd ../

In [73]:
pdf = 'stbl/ociaml3_delta_stbl.pdf'
IFrame(pdf, width=800, height=200)

#### kg1_delta_stbl

In [74]:
study = 'kg1_delta_stbl'

pvmat = ipd.merge_multiple_pvmat(
    pvmat_list = glob(f'stbl/{study}/*c5.go*/pvmatrix.txt') #+ glob(f'stbl/{study}_ind1/*c2.cp.kegg*/pvmatrix.txt')
)

bio_signal = pd.concat([
    ipd.pvmat2bio_signal(pvmat,side='down',n_clust=1),
    ipd.pvmat2bio_signal(pvmat,side='up',n_clust=1),
    ipd.pvmat2bio_signal(pvmat,side='both'),
],axis=0)

bio_signal.to_csv(f'stbl/{study}_pvmatrix.txt',sep='\t')

exp_file = f'{study}.txt'
pvmat_file = f'{study}_pvmatrix.txt'
pdf_file = f'{study}.pdf'

!cd stbl; bash {pagerdir}/ipage_draw_matrix.sh $exp_file $pvmat_file $pdf_file &> /dev/null; cd ../

In [75]:
pdf = 'stbl/kg1_delta_stbl.pdf'
IFrame(pdf, width=800, height=200)

#### thp1_delta_stbl

In [76]:
study = 'thp1_delta_stbl'

pvmat = ipd.merge_multiple_pvmat(
    pvmat_list = glob(f'stbl/{study}/*c5.go*/pvmatrix.txt') #+ glob(f'stbl/{study}_ind1/*c2.cp.kegg*/pvmatrix.txt')
)

bio_signal = pd.concat([
    ipd.pvmat2bio_signal(pvmat,side='down',n_clust=1),
    ipd.pvmat2bio_signal(pvmat,side='up',n_clust=1),
    ipd.pvmat2bio_signal(pvmat,side='both'),
],axis=0)

bio_signal.to_csv(f'stbl/{study}_pvmatrix.txt',sep='\t')

exp_file = f'{study}.txt'
pvmat_file = f'{study}_pvmatrix.txt'
pdf_file = f'{study}.pdf'

!cd stbl; bash {pagerdir}/ipage_draw_matrix.sh $exp_file $pvmat_file $pdf_file &> /dev/null; cd ../

In [77]:
pdf = 'stbl/thp1_delta_stbl.pdf'
IFrame(pdf, width=800, height=200)

# 
___

In [273]:
!date

Tue Feb  8 17:38:38 PST 2022
